In [99]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt
import random
import itertools

In [117]:
def delta(a, b):
    '''
    The Kronecker delta function
    
    a = b  -> return 1
    a != b -> return 0
    '''
    if a == b:
        return 1
    return 0

In [27]:
# NOTE: The number of partcile must be an even integer

numberOfParticles = 8
if numberOfParticles % 2 != 0:
    raise ValueError

In [11]:
# ----------------------------------
# Create the random coupling tensor
# ----------------------------------

random.seed()

N = numberOfParticles

J = np.zeros((N, N, N, N))  # Initialize the tensor with zeros
for a in range(N):          # Insert random numbers to J
    for b in range(N):
        for c in range(N):
            for d in range(N):
                J[a, b, c, d] = random.gauss(0, 6.0 / (N * N * N))

for a in range(N):          # Make the tensor J be totally anti symmetric
    for b in range(N):
        for c in range(N):
            for d in range(N):
                if a == b or a == c or a == d or b == c or b == d or c == d:
                    J[a, b, c, d] = 0
                if a < b < c < d:
                    J[a, b, d, c] = -J[a, b, c, d]
                    J[a, c, b, d] = -J[a, b, c, d]
                    J[a, d, c, b] = -J[a, b, c, d]
                    
                    J[b, a, c, d] = -J[a, b, c, d]
                    J[b, c, d, a] = -J[a, b, c, d]
                    J[b, d, a, c] = -J[a, b, c, d]
                    
                    J[c, a, d, b] = -J[a, b, c, d]
                    J[c, b, a, d] = -J[a, b, c, d]
                    J[c, d, b, a] = -J[a, b, c, d]
                    
                    J[d, a, b, c] = -J[a, b, c, d]
                    J[d, b, c, a] = -J[a, b, c, d]
                    J[d, c, a, b] = -J[a, b, c, d]
                    
                    J[a, c, d, b] = J[a, b, c, d]
                    J[a, d, b, c] = J[a, b, c, d]
                    
                    J[b, a, d, c] = J[a, b, c, d]
                    J[b, c, a, d] = J[a, b, c, d]
                    J[b, d, c, a] = J[a, b, c, d]
                    
                    J[c, a, b, d] = J[a, b, c, d]
                    J[c, b, d, a] = J[a, b, c, d]
                    J[c, d, a, b] = J[a, b, c, d]
                    
                    J[d, a, c, b] = J[a, b, c, d]
                    J[d, b, a, c] = J[a, b, c, d]
                    J[d, c, b, a] = J[a, b, c, d]

In [147]:
#
# Create an array which has state vectors in the Fock space as its entries
# The creation/annihilation opetators is defined by the following:
#     c_i          = (psi_2i - i psi_(2i-1)) / sqrt(2)
#     c_i^dagger   = (psi_2i + i psi_(2i-1)) / sqrt(2)
# where i runs from 1 to the half of the number of particles
#
# For example, if the number of particles is 4 then
# stateVectorArray = [[0,0], [0,1], [1,0], [1,1]]
#

N = int(numberOfParticles / 2)

stateVectorArray = []
stateVector      = [0 for _ in range(N)]

def createAllPatterns(i):
    global stateVectorArray
    global stateVector
    
    if i == N:
        print(stateVector)
        return
    
    createAllPatterns(i + 1)
    stateVector[i] = 1
    createAllPatterns(i + 1)
    stateVector[i] = 0

In [148]:
createAllPatterns(0)

[0, 0, 0, 0]
[0, 0, 0, 1]
[0, 0, 1, 0]
[0, 0, 1, 1]
[0, 1, 0, 0]
[0, 1, 0, 1]
[0, 1, 1, 0]
[0, 1, 1, 1]
[1, 0, 0, 0]
[1, 0, 0, 1]
[1, 0, 1, 0]
[1, 0, 1, 1]
[1, 1, 0, 0]
[1, 1, 0, 1]
[1, 1, 1, 0]
[1, 1, 1, 1]


In [149]:
stateVector

[0, 0, 0, 0]